In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Hyper-parameters of the model
num_epochs = 4
batch_size = 4
learning_rate = 0.003

In [3]:
#I am using https://wandb.ai/site as my performance visualisation tool.
#import wandb
#wandb.init(project="lecture CNN", entity="xxx")
#wandb.config = {
#  "learning_rate": learning_rate,
#  "epochs": num_epochs,
#  "batch_size": batch_size
#}


In [4]:
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
#DOWNLOAD DATASET 
train_dataset = torchvision.datasets.CIFAR10(root = './data_CIFAR10', train=True,
                                            download=False, transform=transform)


test_dataset = torchvision.datasets.CIFAR10(root = './data_CIFAR10', train=False,
                                            download=False, transform=transform)


In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [7]:
# implement conv net
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) #3 channels, 6 filters, 5x5 filter size
        self.pool = nn.MaxPool2d(2,2) #size 2X2, stride: 2
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120) #16*5*5= number of output channels * dimension of the output layer after feature learning; 120 manually set as outcome of the 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) #the output dimension (in our case 10) needs to be equal to the number of classes
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5) #torch.view() function freshape the input tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = ConvNet().to(device)

In [8]:
criterion = nn.CrossEntropyLoss() #CrossEntropyLoss already includes SoftMax
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad() #with zero_grad() we ensure that the gradients are properly reset to zero at the start of each iteration
        loss.backward()
        optimizer.step()
        #wandb.log({"loss": loss})  uncomment this line if you want to send data to weights and biases interface


        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            # Calculate average accuracy for every 2000 steps
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                accuracy = correct / total
                #wandb.log({"accuracy": accuracy}) uncomment this line if you want to send data to weights and biases interface
        
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [1/4], Step [2000/12500], Loss: 2.2965
Epoch [1/4], Step [4000/12500], Loss: 2.1858
Epoch [1/4], Step [6000/12500], Loss: 1.7513
Epoch [1/4], Step [8000/12500], Loss: 2.8124
Epoch [1/4], Step [10000/12500], Loss: 1.8316
Epoch [1/4], Step [12000/12500], Loss: 1.6158
Epoch [2/4], Step [2000/12500], Loss: 1.2682
Epoch [2/4], Step [4000/12500], Loss: 1.8478
Epoch [2/4], Step [6000/12500], Loss: 2.3538
Epoch [2/4], Step [8000/12500], Loss: 1.4884
Epoch [2/4], Step [10000/12500], Loss: 2.1507
Epoch [2/4], Step [12000/12500], Loss: 0.9587
Epoch [3/4], Step [2000/12500], Loss: 1.9258
Epoch [3/4], Step [4000/12500], Loss: 1.3348
Epoch [3/4], Step [6000/12500], Loss: 1.4528
Epoch [3/4], Step [8000/12500], Loss: 1.9277
Epoch [3/4], Step [10000/12500], Loss: 1.5106
Epoch [3/4], Step [12000/12500], Loss: 1.0943
Epoch [4/4], Step [2000/12500], Loss: 1.6769
Epoch [4/4], Step [4000/12500], Loss: 1.2518
Epoch [4/4], Step [6000/12500], Loss: 1.4583
Epoch [4/4], Step [8000/12500], Loss: 1.0744
Epoc

In [10]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 55.74 %
Accuracy of plane: 52.7 %
Accuracy of car: 49.2 %
Accuracy of bird: 28.5 %
Accuracy of cat: 31.1 %
Accuracy of deer: 42.9 %
Accuracy of dog: 63.6 %
Accuracy of frog: 73.8 %
Accuracy of horse: 59.5 %
Accuracy of ship: 82.4 %
Accuracy of truck: 73.7 %
